## Vector-valued function with Random Fourier Features

### Imports

In [461]:
import numpy as np
from numpy.random import randn

### Values

In [462]:

dimension = 2
sigma = 10
sigma2 = 2*sigma**2

# Random Fourier features
m = 20 # number of samples
w = np.array([randn(m) / sigma, randn(m) / sigma]) # w is the same size as the dimensions times the number of samples
b = np.random.rand(m)*2*np.pi # b is the same size as number of samples

n = 100 # number of points

x1 = randn(n)/2
x2 =  randn(n)/2
x_vector = np.array([x1, x2])
# x_vector = np.block([x1,x2])

y1 = np.cos(x1) + randn(n) * 1e-1
y2 = np.sin(x2) + randn(n) * 1e-1


#y1 = 1+np.sin(x1/10) + x1**2
#y2 = 0.5 * x2**2 + np.exp(x2)

y = np.block([y1, y2])
print(y)

#print(y1, y2)
lam = 0.000001 # Regularisaton parameter

[1.02068273 2.22770149 1.04879896 1.06302667 1.00152865 1.82566987
 1.07332823 1.03306774 1.0814398  1.19227231 1.23725775 1.55697134
 1.3714793  1.28236405 1.12731949 0.99923679 1.08408141 1.01219655
 1.00364753 1.49423339 1.00245975 1.12219969 0.99774289 1.21473213
 1.2946659  1.00489642 1.16566998 0.99750418 2.10942162 1.00355744
 1.18222551 1.57742764 0.99899665 1.27292754 1.01620598 1.63734201
 0.99766161 1.11023716 1.09838216 1.30668428 1.13818984 1.53480774
 2.46339493 0.99867906 1.01402709 1.01351431 1.01742499 1.03469299
 0.99844617 1.15524871 1.04105603 1.00224387 1.05281889 1.03292722
 0.99770181 1.03653902 1.22645503 0.9992398  1.16933129 1.30982849
 1.73084591 1.17749958 1.02551218 1.84307059 1.5229438  1.19642527
 1.6989666  1.07682126 1.84112546 0.99869883 1.02212287 1.40012669
 1.09968457 1.00716034 1.20086222 0.9976227  0.99760446 0.99895738
 1.03619082 1.15233182 1.21468798 1.42231784 0.99852837 0.99772403
 1.00651269 1.16086001 1.11586359 1.01314485 0.99751898 1.2180

### Gaussian Kernel

In [463]:
def k_gauss(x, z):
    k = np.exp(-((np.linalg.norm(x-z))**2)/(sigma2))
    return k

def k_matrix(x, z, dim):
    k = k_gauss(x, z)
    k_matrix = k *np.eye(dim)
    return k_matrix

def K_gauss(x, n, dim):
    K = np.zeros((dim*n, dim*n))
    for i in range(0, n):
        for j in range(0, n):
            k = k_gauss(x[:,i], x[:,j])
            K[i*dim][j*dim] = k
            K[i*dim+1][j*dim+1] = k
    return K

K = K_gauss(x_vector, n, dimension)
print(K)

[[1.         0.         0.99456373 ... 0.         0.99835879 0.        ]
 [0.         1.         0.         ... 0.99975128 0.         0.99835879]
 [0.99456373 0.         1.         ... 0.         0.99500219 0.        ]
 ...
 [0.         0.99975128 0.         ... 1.         0.         0.99759961]
 [0.99835879 0.         0.99500219 ... 0.         1.         0.        ]
 [0.         0.99835879 0.         ... 0.99759961 0.         1.        ]]


#### Alpha

In [464]:
# Returns a vector with length dim * n
def alpha(x, y, lam, n, dim):
    K = K_gauss(x, n, dim)
    alpha_vector = np.linalg.inv((K + n*lam*np.eye(dim*n))) @ y
    alpha_reshape = [alpha_vector[::2], alpha_vector[1::2]]
    return np.array(alpha_reshape)

alpha_ = alpha(x_vector, y, lam, n, dimension)
print(alpha_)

[[-2944.32160453   342.02318969 -3577.06875424 -3150.12480712
  -3074.71954335   961.55914752  -141.71856287 -1943.57917276
   -975.91121648 -3476.14059881 -3189.25135758 -2193.31483346
    433.48241789 -1179.31620556  7657.08659126  -856.98307737
  -2459.84351413 -3445.02064147 -3661.06235001 -1969.32897036
  -2576.64287013 11631.87677802 -3378.63488718 -3091.19040903
  -3347.56495931 -2594.93158236 -2824.65592543 -3962.57341667
   1619.13676721 -2266.72078952  3642.06453698 -1839.26588426
   1279.41482054  3489.99613508  4765.23962969   377.89379003
    733.80524249  -379.86992887 -3602.56964803 -2407.72916713
  -1370.83212204 -2381.52645241   196.45374592 -2403.97640012
  -2562.60786298 -2835.30170521 -1598.35269142   889.82050229
  15482.98956387 -2437.48321368 12097.1640709   2301.75742586
  -3307.30953889  1934.39043201 -4804.44872836 -3428.93501797
  -6235.46040365 -4825.1214423  -6077.88274797  8919.85853422
   2281.56848406 10255.31557662  8708.78390773 -2862.18167406
   4635.

#### Estimated function

In [465]:
def estiamted_function(alpha, x, n, dim):
    function = np.zeros((2, n))
    for i in range(n):
        sum1 = np.zeros((2))
        x_i = x[:, i]
        for j in range(n):
            k = k_gauss(x_i, x[:, j])
            kernel = k*np.eye(dim)
            sum1 = np.add(sum1, kernel@alpha[:, j])
        function[:, i] = sum1
    return function

estiamted_function(alpha_, x_vector, n, dimension)


array([[1.31511489, 1.01459664, 1.35923553, 1.38834071, 1.38891176,
        1.14110183, 1.38565115, 1.32167741, 1.18167253, 1.35126159,
        1.32138489, 1.21707437, 1.25131766, 1.2836016 , 1.34371296,
        1.26792382, 1.244981  , 1.36070804, 1.36376785, 1.29531505,
        1.39585413, 1.30020726, 1.35189058, 1.32654403, 1.33320267,
        1.30054919, 1.33528448, 1.39395916, 1.06454136, 1.39600337,
        1.36663945, 1.20943877, 1.39500232, 1.34996698, 1.36460149,
        0.98433349, 1.02630405, 1.23884921, 1.35786143, 1.27696373,
        1.35177119, 1.23668101, 0.98686732, 1.35626123, 1.25377976,
        1.28513198, 1.25339228, 1.3745682 , 1.36198496, 1.37183191,
        1.35016184, 1.39443605, 1.34207788, 1.37902203, 1.33902326,
        1.3384256 , 1.35575784, 1.34978996, 1.33978172, 1.32589581,
        1.10518337, 1.32263372, 1.37418763, 1.16500123, 0.90568889,
        1.32841869, 1.18669309, 1.36160362, 1.14739595, 1.30666179,
        1.3773113 , 0.98453038, 1.35735608, 1.37

#### Plot

### With Random Fourier Features

In [472]:
def psi(x, w, b):
    return np.sqrt(2) * np.cos(w.T@x + b)

def psi_vector(x, w, b, m):
    psi_vector = np.zeros(m)
    for i in range(m):
        psi_vector[i] = psi(x[:, i], w[:, i], b[i])
    return np.array(psi_vector)

def capital_psi(psi_vector, dim):
    return np.kron(psi_vector, dim)

psi_ = psi_vector(x_vector, w, b, m)
capital_psi(psi_, dimension)

Vektor  [-0.76746819  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
Vektor  [-0.76746819  1.41259723  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
Vektor  [-0.76746819  1.41259723 -1.38540841  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
Vektor  [-0.76746819  1.41259723 -1.38540841  0.56428052  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
Vektor  [-0.76746819  1.41259723 -1.38540841  0.56428052  0.91416392  0.
  0.       

array([-1.53493638,  2.82519446, -2.77081682,  1.12856103,  1.82832783,
        2.50913341,  2.33264247, -0.90230623,  2.8231739 , -2.67433046,
       -0.46032965,  2.64520108,  0.94317817, -2.68998329,  2.6173241 ,
        0.88460113,  0.41934461,  2.76778898,  1.70830899,  2.62389263])